In [ ]:
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
%run split_data.ipynb

In [ ]:
vgg = models.vgg16(pretrained=True)

for i, (name, param) in enumerate(vgg.features.named_parameters()):
    param.requires_grad = False


trainable_layers = [21, 24, 26, 28]
for i in trainable_layers:
    for param in vgg.features[i].parameters():
        param.requires_grad = True
    
for param in vgg.classifier.parameters():
    param.requires_grad = True


num_classes = 105  
vgg.classifier[-1] = torch.nn.Linear(in_features=4096, out_features=num_classes)



criterion = nn.CrossEntropyLoss()
learning_rate = 0.008
layer_lr = [
    {'params': vgg.features.parameters(), 'lr': learning_rate / 10},
    {'params': vgg.classifier.parameters(), 'lr': learning_rate}
]

optimizer = optim.SGD(layer_lr, lr=learning_rate, momentum=0.9)


num_epochs = 30
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg.to(device)


def train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=30):
    train_loss_history = []
    valid_loss_history = []
    train_acc_history = []
    valid_acc_history = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / total
        epoch_acc = running_corrects.double() / total

        train_loss_history.append(epoch_loss)
        train_acc_history.append(epoch_acc.item())

        model.eval()
        running_loss = 0.0
        running_corrects = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                total += labels.size(0)

            epoch_loss = running_loss / total
            epoch_acc = running_corrects.double() / total

            valid_loss_history.append(epoch_loss)
            valid_acc_history.append(epoch_acc.item())

        print(f'Epoch {epoch}/{num_epochs - 1}')
        print(f'Train Loss: {train_loss_history[-1]:.4f} Acc: {train_acc_history[-1]:.4f}')
        print(f'Valid Loss: {valid_loss_history[-1]:.4f} Acc: {valid_acc_history[-1]:.4f}')

    return train_loss_history, valid_loss_history, train_acc_history, valid_acc_history

In [ ]:
train_directory = 'trening'
train_dataloader = load_images_and_create_dataloader(train_directory, batch_size=32)
validate_directory = 'validate'
validate_dataloader = load_images_and_create_dataloader(validate_directory, batch_size=32)

In [ ]:
train_loss, valid_loss, train_acc, valid_acc = train_model(vgg, criterion, optimizer, train_dataloader, validate_dataloader, num_epochs=num_epochs)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(num_epochs), train_loss, label='Training Loss')
plt.plot(range(num_epochs), valid_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss VGG')
plt.savefig('training_validation_loss_vgg1.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(num_epochs), train_acc, label='Training Accuracy')
plt.plot(range(num_epochs), valid_acc, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training and Validation Accuracy VGG')
plt.savefig('training_validation_accuracy_vgg1.png')
plt.show()

In [ ]:
save_model(vgg, "vgg_fc_and_conv_layers.pth")